# Chapter 6

# Section 6.3.2 Subsample Analysis

# VAR-BEKK-GARCH model for Period 4 (2020/2 - 2024/5) (Section 6.3.2)

In [1]:
library(xts)
library(BEKKs)
library(MTS)
library(vars)
library(stargazer)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘MTS’


The following object is masked from ‘package:BEKKs’:

    backtest


Loading required package: MASS

Loading required package: strucchange

Loading required package: sandwich

Loading required package: urca

Loading required package: lmtest


Attaching package: ‘vars’


The following object is masked from ‘package:MTS’:

    VAR



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




# Dataset Preperation

In [2]:
fr_d <- readRDS("fr_d.Rds")
fp_d <- readRDS("fp_d.Rds")
cr_d <- readRDS("cr_d.Rds")
cp_d <- readRDS("cp_d.Rds")
head(cr_d)
head(fr_d)
us_d <- cr_d[,1]
l_d <- cr_d[,2]

                     usc           lc
2007-07-06  0.0033026687  0.005272420
2007-07-09 -0.0108928380 -0.013233541
2007-07-10 -0.0378502241 -0.040784991
2007-07-11  0.0210431058  0.019240172
2007-07-12  0.0057943185  0.004526943
2007-07-13 -0.0009633912 -0.004526943

                  eurusd        gbpusd        chfusd        usdgbp       eurgbp
2007-07-06  2.240547e-03 -0.0004972527 -0.0003044418  0.0007037652  0.002659380
2007-07-09  7.337565e-05  0.0022605055  0.0012172115 -0.0025157246 -0.001993871
2007-07-10  8.584371e-03  0.0057647029  0.0115503355 -0.0056582954  0.002805051
2007-07-11 -7.274844e-05  0.0027838359 -0.0029504763 -0.0026379885 -0.002952903
2007-07-12  3.485969e-03 -0.0009845428  0.0028903495  0.0008124302  0.004573291
2007-07-13 -7.252421e-04  0.0019435396  0.0000000000 -0.0020323145 -0.003391083
                  chfgbp        ghsusd        ghsgbp
2007-07-06  0.0001273483 -0.0033264938 -2.614079e-03
2007-07-09 -0.0009873737  0.0040086898  1.236683e-03
2007-07-10  0.0059364761 -0.0002705906 -5.801912e-03
2007-07-11 -0.0057330414  0.0026887042  4.994937e-05
2007-07-12  0.0037670817  0.0027212691  3.716576e-03
2007-07-13 -0.0020261005 -0.0051426368 -7.210559e-03

In [3]:
d <- na.omit(merge(cr_d,fr_d))
colnames(d) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

In [4]:
d_p <- na.omit(merge(cp_d,fp_d))
colnames(d_p) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

# Period 4: Data 2020/2 - 2024/5

In [5]:
d <- d["2020-2/2024-5"]
str(d)
head(d)
tail(d)

An xts object on 2020-02-03 / 2024-05-03 containing: 
  Data:    double [1049, 10]
  Columns: US Cocoa, London Cocoa, EURUSD, GBPUSD, CHFUSD ... with 5 more columns
  Index:   Date [1049] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


               US Cocoa London Cocoa       EURUSD       GBPUSD        CHFUSD
2020-02-03 -0.016196143 -0.005089070 -0.003069426 -0.015727916 -0.0027972046
2020-02-04  0.009030222  0.000000000 -0.001447702  0.002728095 -0.0034833127
2020-02-05  0.012151686  0.019202207 -0.004082934 -0.002189572 -0.0042739259
2020-02-06  0.017606089  0.015889113 -0.001683387 -0.005552999 -0.0011687934
2020-02-07  0.011452494  0.007361996 -0.003238241 -0.002904446 -0.0033679104
2020-02-10 -0.006925235 -0.002447982 -0.003111276  0.001743680  0.0006354017
                 USDGBP       EURGBP        CHFGBP       GHSUSD       GHSGBP
2020-02-03  0.015717416  0.012889672  0.0129490487  0.009388869  0.025123068
2020-02-04 -0.002732775 -0.004120804 -0.0062956642  0.008350745  0.005584836
2020-02-05  0.002212822 -0.002007677 -0.0020230124 -0.003785183 -0.001560331
2020-02-06  0.005575379  0.004129069  0.0043570302 -0.002762874  0.002799130
2020-02-07  0.002905008  0.000000000 -0.0004411533  0.002075578  0.004971314

              US Cocoa London Cocoa       EURUSD        GBPUSD       CHFUSD
2024-04-26 -0.01962863  -0.02951879 -0.003314121 -0.0018396326 -0.002191682
2024-04-29 -0.17075943  -0.15652942  0.002521598  0.0057476010  0.004059947
2024-04-30  0.03865640   0.03216783 -0.005190202 -0.0057476010 -0.009744124
2024-05-01 -0.11385883  -0.08952546  0.004257018  0.0028779299  0.004128825
2024-05-02 -0.09105801  -0.10077335  0.001492677  0.0005586369  0.005524245
2024-05-03  0.07413631   0.04940746  0.003071627  0.0009967904  0.005493896
                  USDGBP        EURGBP        CHFGBP       GHSUSD        GHSGBP
2024-04-26  0.0018128971 -0.0015173624 -0.0003996004 -0.003682821 -0.0018400829
2024-04-29 -0.0056998056 -0.0031588210 -0.0016571906 -0.004412793 -0.0101599929
2024-04-30  0.0056998056  0.0004686036 -0.0040688901 -0.001468434  0.0042437477
2024-05-01 -0.0028145248  0.0018722215  0.0013772526 -0.002189577 -0.0050328783
2024-05-02 -0.0006265272  0.0001168975  0.0049196369 -0.003637066 -0

# VAR-BEKK Models for the subsample

#### The estimation of the VAR-BEKK model is conducted in two stages, VAR model is the model specification for the mean equation (first stage) and BEKK is the specification for the variance equation (second stage).

## (i) Spillover between US Cocoa and London Cocoa

In [6]:
cr_d <- cr_d["2020-2/2024-5"]
str(cr_d)
head(cr_d)
tail(cr_d)

An xts object on 2020-02-03 / 2024-05-03 containing: 
  Data:    double [1049, 2]
  Columns: usc, lc
  Index:   Date [1049] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:159] 37 42 100 101 122 123 140 160 184 214 ...
  ..- attr(*, "index")= num [1:159] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                    usc           lc
2020-02-03 -0.016196143 -0.005089070
2020-02-04  0.009030222  0.000000000
2020-02-05  0.012151686  0.019202207
2020-02-06  0.017606089  0.015889113
2020-02-07  0.011452494  0.007361996
2020-02-10 -0.006925235 -0.002447982

                   usc          lc
2024-04-26 -0.01962863 -0.02951879
2024-04-29 -0.17075943 -0.15652942
2024-04-30  0.03865640  0.03216783
2024-05-01 -0.11385883 -0.08952546
2024-05-02 -0.09105801 -0.10077335
2024-05-03  0.07413631  0.04940746

In [7]:
VARselect(cr_d)

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.776818e+01,-1.776224e+01,-1.775684e+01,-1.776088e+01,-1.775943e+01,-1.775837e+01,-1.775555e+01,-1.775300e+01,-1.775008e+01,-1.774590e+01
HQ(n),-1.775734e+01,-1.774418e+01,-1.773156e+01,-1.772837e+01,-1.771970e+01,-1.771142e+01,-1.770138e+01,-1.769160e+01,-1.768145e+01,-1.767005e+01
SC(n),-1.773962e+01,-1.771464e+01,-1.769020e+01,-1.767519e+01,-1.765471e+01,-1.763460e+01,-1.761274e+01,-1.759115e+01,-1.756918e+01,-1.754596e+01
FPE(n),1.920341e-08,1.931780e-08,1.942237e-08,1.934409e-08,1.937208e-08,1.939270e-08,1.944747e-08,1.949726e-08,1.955430e-08,1.963630e-08


In [8]:
var_cr<-VAR(cr_d,p = 1,type = "const")
summary(var_cr)
res_cr <- residuals(var_cr)
res_cr <- as.data.frame(res_cr)


VAR Estimation Results:
Endogenous variables: usc, lc 
Deterministic variables: const 
Sample size: 1048 
Log Likelihood: 6347.742 
Roots of the characteristic polynomial:
0.07616 0.0106
Call:
VAR(y = cr_d, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + lc.l1 + const 

        Estimate Std. Error t value Pr(>|t|)  
usc.l1 0.0319187  0.0672790   0.474   0.6353  
lc.l1  0.0189529  0.0745933   0.254   0.7995  
const  0.0010109  0.0005664   1.785   0.0746 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01829 on 1045 degrees of freedom
Multiple R-Squared: 0.002246,	Adjusted R-squared: 0.0003361 
F-statistic: 1.176 on 2 and 1045 DF,  p-value: 0.3089 


Estimation results for equation lc: 
lc = usc.l1 + lc.l1 + const 

        Estimate Std. Error t value Pr(>|t|)  
usc.l1 0.0497612  0.0602129   0.826    0.409  
lc.l1  0.0548441  0.0667590   0.822    0.412  
const  0.0010172  0.0005069   2.007    0.045 *
---
Sign

In [9]:
ind_var <- index(cr_d)[-1]
str(ind_var)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [10]:
res_cr <- xts(res_cr,order.by = ind_var)
str(res_cr)
head(res_cr)

An xts object on 2020-02-04 / 2024-05-03 containing: 
  Data:    double [1048, 2]
  Columns: usc, lc
  Index:   Date [1048] (TZ: "UTC")


                    usc            lc
2020-02-04  0.008632741  6.788391e-05
2020-02-05  0.010852560  1.773569e-02
2020-02-06  0.015843392  1.321414e-02
2020-02-07  0.009578492  4.597311e-03
2020-02-10 -0.008441209 -4.438795e-03
2020-02-11 -0.001438621 -2.992285e-03

In [11]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk_fit <- bekk_fit(spec = bekk_spec,data = res_cr)

In [12]:
bekk_fit$A
bekk_fit$A_t
bekk_fit$G
bekk_fit$G_t

0.07167476,-0.08274346
0.20230714,0.41419341


1.412794,-1.841455
3.009310,7.312562


1.0081234,0.03946405
-0.0504013,0.88868695


67.502756,2.199507
-2.063715,32.817876


### VAR-BEKK Model Table for US Cocoa - London Cocoa

In [13]:
bekkc <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_cr)$varresult$usc$coefficients["const","Estimate"],summary(var_cr)$varresult$lc$coefficients["const","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Estimate"],
                                   bekk_fit$A[1,1],bekk_fit$A[1,2],bekk_fit$A[2,1],bekk_fit$A[2,2],bekk_fit$G[1,1],bekk_fit$G[1,2],bekk_fit$G[2,1],bekk_fit$G[2,2]),
                   Std_error = c(summary(var_cr)$varresult$usc$coefficients["const","Std. Error"],summary(var_cr)$varresult$lc$coefficients["const","Std. Error"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Std. Error"],
                                 summary(var_cr)$varresult$lc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Std. Error"],
                                 bekk_fit$A_sd[1,1],bekk_fit$A_sd[1,2],bekk_fit$A_sd[2,1],bekk_fit$A_sd[2,2],bekk_fit$G_sd[1,1],bekk_fit$G_sd[1,2],bekk_fit$G_sd[2,1],bekk_fit$G_sd[2,2]),
                   T_value = c(summary(var_cr)$varresult$usc$coefficients["const","t value"],summary(var_cr)$varresult$lc$coefficients["const","t value"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l1","t value"],
                               summary(var_cr)$varresult$lc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l1","t value"],
                               bekk_fit$A_t[1,1],bekk_fit$A_t[1,2],bekk_fit$A_t[2,1],bekk_fit$A_t[2,2],bekk_fit$G_t[1,1],bekk_fit$G_t[1,2],
                                 bekk_fit$G_t[2,1],bekk_fit$G_t[2,2]))

bekkc$Significant_at_5p_level[abs(bekkc$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.96 & abs(bekkc$T_value) < 2.58] <- "Stat significant at 5%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.645 & abs(bekkc$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) <= 1.645] <- "Not Statistically Significant"
bekkc

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.001010893,0.0005663626,1.7848874,Stat significant at 10%
mu20,0.001017161,0.0005068792,2.0067130,Stat significant at 5%
phi11,0.031918692,0.0672789663,0.4744230,Not Statistically Significant
phi12,0.018952908,0.0745933230,0.2540832,Not Statistically Significant
phi21,0.049761215,0.0602128552,0.8264218,Not Statistically Significant
phi22,0.054844064,0.0667590065,0.8215231,Not Statistically Significant
a11,0.071674760,0.0507326304,1.4127941,Not Statistically Significant
a12,-0.082743458,0.0449337308,-1.8414553,Stat significant at 10%
a21,0.202307139,0.0672270790,3.0093103,Stat significant at 1%


In [14]:
causality(var_cr,cause = "usc")$Granger
causality(var_cr,cause = "lc")$Granger


	Granger causality H0: usc do not Granger-cause lc

data:  VAR object var_cr
F-Test = 0.68297, df1 = 1, df2 = 2090, p-value = 0.4087



	Granger causality H0: lc do not Granger-cause usc

data:  VAR object var_cr
F-Test = 0.064558, df1 = 1, df2 = 2090, p-value = 0.7995


## (ii) VAR-BEKK Models for US Cocoa

In [15]:
us_d <- na.omit(merge(cr_d[,1],fr_d))
str(us_d)
head(us_d)
tail(us_d)

An xts object on 2020-02-03 / 2024-05-03 containing: 
  Data:    double [1049, 9]
  Columns: usc, eurusd, gbpusd, chfusd, usdgbp ... with 4 more columns
  Index:   Date [1049] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3342] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3342] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                    usc       eurusd       gbpusd        chfusd       usdgbp
2020-02-03 -0.016196143 -0.003069426 -0.015727916 -0.0027972046  0.015717416
2020-02-04  0.009030222 -0.001447702  0.002728095 -0.0034833127 -0.002732775
2020-02-05  0.012151686 -0.004082934 -0.002189572 -0.0042739259  0.002212822
2020-02-06  0.017606089 -0.001683387 -0.005552999 -0.0011687934  0.005575379
2020-02-07  0.011452494 -0.003238241 -0.002904446 -0.0033679104  0.002905008
2020-02-10 -0.006925235 -0.003111276  0.001743680  0.0006354017 -0.001741992
                 eurgbp        chfgbp       ghsusd       ghsgbp
2020-02-03  0.012889672  0.0129490487  0.009388869  0.025123068
2020-02-04 -0.004120804 -0.0062956642  0.008350745  0.005584836
2020-02-05 -0.002007677 -0.0020230124 -0.003785183 -0.001560331
2020-02-06  0.004129069  0.0043570302 -0.002762874  0.002799130
2020-02-07  0.000000000 -0.0004411533  0.002075578  0.004971314
2020-02-10 -0.005311941 -0.0010721833  0.000589913 -0.001110366

                   usc       eurusd        gbpusd       chfusd        usdgbp
2024-04-26 -0.01962863 -0.003314121 -0.0018396326 -0.002191682  0.0018128971
2024-04-29 -0.17075943  0.002521598  0.0057476010  0.004059947 -0.0056998056
2024-04-30  0.03865640 -0.005190202 -0.0057476010 -0.009744124  0.0056998056
2024-05-01 -0.11385883  0.004257018  0.0028779299  0.004128825 -0.0028145248
2024-05-02 -0.09105801  0.001492677  0.0005586369  0.005524245 -0.0006265272
2024-05-03  0.07413631  0.003071627  0.0009967904  0.005493896 -0.0009405274
                  eurgbp        chfgbp       ghsusd        ghsgbp
2024-04-26 -0.0015173624 -0.0003996004 -0.003682821 -0.0018400829
2024-04-29 -0.0031588210 -0.0016571906 -0.004412793 -0.0101599929
2024-04-30  0.0004686036 -0.0040688901 -0.001468434  0.0042437477
2024-05-01  0.0018722215  0.0013772526 -0.002189577 -0.0050328783
2024-05-02  0.0001168975  0.0049196369 -0.003637066 -0.0041962674
2024-05-03  0.0025682947  0.0044411620  0.001449549  0.0004565954

## (a) US Cocoa - EURUSD

In [16]:
bekk1data <- us_d[,c(1:2)]
head(bekk1data)
tail(bekk1data)
VARselect(bekk1data,type = "const")

                    usc       eurusd
2020-02-03 -0.016196143 -0.003069426
2020-02-04  0.009030222 -0.001447702
2020-02-05  0.012151686 -0.004082934
2020-02-06  0.017606089 -0.001683387
2020-02-07  0.011452494 -0.003238241
2020-02-10 -0.006925235 -0.003111276

                   usc       eurusd
2024-04-26 -0.01962863 -0.003314121
2024-04-29 -0.17075943  0.002521598
2024-04-30  0.03865640 -0.005190202
2024-05-01 -0.11385883  0.004257018
2024-05-02 -0.09105801  0.001492677
2024-05-03  0.07413631  0.003071627

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.862758e+01,-1.862035e+01,-1.861588e+01,-1.861544e+01,-1.860836e+01,-1.862133e+01,-1.861815e+01,-1.861504e+01,-1.860781e+01,-1.860447e+01
HQ(n),-1.861674e+01,-1.860230e+01,-1.859060e+01,-1.858293e+01,-1.856863e+01,-1.857438e+01,-1.856397e+01,-1.855363e+01,-1.853919e+01,-1.852862e+01
SC(n),-1.859901e+01,-1.857275e+01,-1.854924e+01,-1.852975e+01,-1.850363e+01,-1.849756e+01,-1.847534e+01,-1.845318e+01,-1.842692e+01,-1.840454e+01
FPE(n),8.131047e-09,8.189981e-09,8.226708e-09,8.230375e-09,8.288825e-09,8.182013e-09,8.208114e-09,8.233704e-09,8.293414e-09,8.321196e-09


In [17]:
var_bekk1<-VAR(bekk1data,p = 1,type = "const")
res_bekk1 <- residuals(var_bekk1)
res_bekk1 <- as.data.frame(res_bekk1)

In [18]:
summary(var_bekk1)


VAR Estimation Results:
Endogenous variables: usc, eurusd 
Deterministic variables: const 
Sample size: 1048 
Log Likelihood: 6799.262 
Roots of the characteristic polynomial:
0.05156 0.02646
Call:
VAR(y = bekk1data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + eurusd.l1 + const 

           Estimate Std. Error t value Pr(>|t|)  
usc.l1    0.0437889  0.0312342   1.402   0.1612  
eurusd.l1 0.1341719  0.1156576   1.160   0.2463  
const     0.0010234  0.0005656   1.809   0.0707 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01828 on 1045 degrees of freedom
Multiple R-Squared: 0.003467,	Adjusted R-squared: 0.00156 
F-statistic: 1.818 on 2 and 1045 DF,  p-value: 0.1629 


Estimation results for equation eurusd: 
eurusd = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     1.003e-03  8.384e-03   0.120    0.905
eurusd.l1  3.423e-02  3.104e-02   1.103    0.270
const     -2.

In [19]:
ind_var1 <- index(bekk1data)[-1]
str(ind_var1)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [20]:
res_bekk1 <- xts(res_bekk1,order.by = ind_var1)

In [21]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk1_fit <- bekk_fit(spec = bekk_spec,data = res_bekk1)

In [22]:
bekk1_fit$A
bekk1_fit$A_t
bekk1_fit$G
bekk1_fit$G_t

0.2178314,0.002855132
-0.1101271,0.190915921


10.498378,0.4307619
-1.256734,8.4426248


0.97469772,-0.00132217
0.02361825,0.97559551


143.7002302,-0.5611507
0.9517062,152.0906528


### VAR-BEKK Model Table for US Cocoa - EURUSD

In [23]:
bekk1 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk1)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Estimate"],
                                   summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Estimate"],
                                   summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Estimate"],
                                   bekk1_fit$A[1,1],bekk1_fit$A[1,2],bekk1_fit$A[2,1],bekk1_fit$A[2,2],bekk1_fit$G[1,1],bekk1_fit$G[1,2],bekk1_fit$G[2,1],bekk1_fit$G[2,2]),
                   Std_error = c(summary(var_bekk1)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Std. Error"],
                                summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Std. Error"],
                                 summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Std. Error"],
                                 bekk1_fit$A_sd[1,1],bekk1_fit$A_sd[1,2],bekk1_fit$A_sd[2,1],bekk1_fit$A_sd[2,2],bekk1_fit$G_sd[1,1],bekk1_fit$G_sd[1,2],bekk1_fit$G_sd[2,1],bekk1_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk1)$varresult$usc$coefficients["const","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["const","t value"],
                       summary(var_bekk1)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","t value"],
                               summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","t value"],
                               bekk1_fit$A_t[1,1],bekk1_fit$A_t[1,2],bekk1_fit$A_t[2,1],bekk1_fit$A_t[2,2],bekk1_fit$G_t[1,1],bekk1_fit$G_t[1,2],bekk1_fit$G_t[2,1],bekk1_fit$G_t[2,2]))

bekk1$Significant_at_5p_level[abs(bekk1$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.96 & abs(bekk1$T_value) < 2.58] <- "Stat significant at 5%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.645 & abs(bekk1$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) <= 1.645] <- "Not Statistically Significant"
bekk1

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.023408e-03,0.0005656291,1.8093269,Stat significant at 10%
mu20,-2.228401e-05,0.0001518210,-0.1467782,Not Statistically Significant
phi11,4.378894e-02,0.0312342110,1.4019543,Not Statistically Significant
phi12,1.341719e-01,0.1156576082,1.1600787,Not Statistically Significant
phi21,1.003319e-03,0.0083835999,0.1196764,Not Statistically Significant
phi22,3.422702e-02,0.0310437524,1.1025413,Not Statistically Significant
a11,2.178314e-01,0.0207490484,10.4983782,Stat significant at 1%
a12,2.855132e-03,0.0066280986,0.4307619,Not Statistically Significant
a21,-1.101271e-01,0.0876296707,-1.2567335,Not Statistically Significant


In [24]:
causality(var_bekk1,cause = "usc")$Granger
causality(var_bekk1,cause = "eurusd")$Granger


	Granger causality H0: usc do not Granger-cause eurusd

data:  VAR object var_bekk1
F-Test = 0.014322, df1 = 1, df2 = 2090, p-value = 0.9048



	Granger causality H0: eurusd do not Granger-cause usc

data:  VAR object var_bekk1
F-Test = 1.3458, df1 = 1, df2 = 2090, p-value = 0.2461


## (b) VAR-BEKK: US Cocoa-GBPUSD

In [25]:
bekk2data <- us_d[,c(1,3)]
head(bekk2data)
tail(bekk2data)

                    usc       gbpusd
2020-02-03 -0.016196143 -0.015727916
2020-02-04  0.009030222  0.002728095
2020-02-05  0.012151686 -0.002189572
2020-02-06  0.017606089 -0.005552999
2020-02-07  0.011452494 -0.002904446
2020-02-10 -0.006925235  0.001743680

                   usc        gbpusd
2024-04-26 -0.01962863 -0.0018396326
2024-04-29 -0.17075943  0.0057476010
2024-04-30  0.03865640 -0.0057476010
2024-05-01 -0.11385883  0.0028779299
2024-05-02 -0.09105801  0.0005586369
2024-05-03  0.07413631  0.0009967904

In [26]:
VARselect(bekk2data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.821374e+01,-1.820721e+01,-1.820097e+01,-1.819683e+01,-1.819579e+01,-1.819999e+01,-1.819692e+01,-1.819258e+01,-1.818790e+01,-1.818347e+01
HQ(n),-1.820291e+01,-1.818915e+01,-1.817568e+01,-1.816432e+01,-1.815606e+01,-1.815304e+01,-1.814274e+01,-1.813118e+01,-1.811928e+01,-1.810763e+01
SC(n),-1.818518e+01,-1.815960e+01,-1.813432e+01,-1.811114e+01,-1.809106e+01,-1.807622e+01,-1.805411e+01,-1.803073e+01,-1.800701e+01,-1.798354e+01
FPE(n),1.229909e-08,1.237972e-08,1.245723e-08,1.250886e-08,1.252188e-08,1.246936e-08,1.250776e-08,1.256213e-08,1.262112e-08,1.267716e-08


In [27]:
var_bekk2<-VAR(bekk2data,p = 1,type = "const")
res_bekk2 <- residuals(var_bekk2)
res_bekk2 <- as.data.frame(res_bekk2)

In [28]:
summary(var_bekk2)


VAR Estimation Results:
Endogenous variables: usc, gbpusd 
Deterministic variables: const 
Sample size: 1048 
Log Likelihood: 6582.088 
Roots of the characteristic polynomial:
0.1047 0.01722
Call:
VAR(y = bekk2data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + gbpusd.l1 + const 

          Estimate Std. Error t value Pr(>|t|)  
usc.l1    0.039438   0.031322   1.259   0.2083  
gbpusd.l1 0.179818   0.093276   1.928   0.0542 .
const     0.001030   0.000565   1.822   0.0687 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01826 on 1045 degrees of freedom
Multiple R-Squared: 0.00572,	Adjusted R-squared: 0.003817 
F-statistic: 3.006 on 2 and 1045 DF,  p-value: 0.04992 


Estimation results for equation gbpusd: 
gbpusd = usc.l1 + gbpusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)   
usc.l1     8.061e-03  1.040e-02   0.775  0.43833   
gbpusd.l1  8.248e-02  3.096e-02   2.664  0.00785 **
const   

In [29]:
ind_var2 <- index(bekk2data)[-1]
str(ind_var2)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [30]:
res_bekk2 <- xts(res_bekk2,order.by = ind_var2)

In [31]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk2_fit <- bekk_fit(spec = bekk_spec,data = res_bekk2)

In [32]:
bekk2_fit$A
bekk2_fit$A_t
bekk2_fit$G
bekk2_fit$G_t

0.19915716,0.003922008
-0.02435078,0.286491245


11.8517495,0.4051778
-0.3067693,12.2953591


0.979723666,-0.001557959
-0.002154775,0.939936593


194.1532020,-0.497345
-0.0688213,89.104414


### VAR-BEKK Model Table for US Cocoa - GBPUSD

In [33]:
bekk2 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk2)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Estimate"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Estimate"],
                                   bekk2_fit$A[1,1],bekk2_fit$A[1,2],bekk2_fit$A[2,1],bekk2_fit$A[2,2],bekk2_fit$G[1,1],bekk2_fit$G[1,2],bekk2_fit$G[2,1],bekk2_fit$G[2,2]),
                   Std_error = c(summary(var_bekk2)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Std. Error"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Std. Error"],
                                 summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Std. Error"],
                                 bekk2_fit$A_sd[1,1],bekk2_fit$A_sd[1,2],bekk2_fit$A_sd[2,1],bekk2_fit$A_sd[2,2],bekk2_fit$G_sd[1,1],bekk2_fit$G_sd[1,2],bekk2_fit$G_sd[2,1],bekk2_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk2)$varresult$usc$coefficients["const","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","t value"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","t value"],
                               summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","t value"],
                               bekk2_fit$A_t[1,1],bekk2_fit$A_t[1,2],bekk2_fit$A_t[2,1],bekk2_fit$A_t[2,2],bekk2_fit$G_t[1,1],bekk2_fit$G_t[1,2],bekk2_fit$G_t[2,1],bekk2_fit$G_t[2,2]))

bekk2$Significant_at_5p_level[abs(bekk2$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.96 & abs(bekk2$T_value) < 2.58] <- "Stat significant at 5%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.645 & abs(bekk2$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) <= 1.645] <- "Not Statistically Significant"
bekk2

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0010296977,0.0005649994,1.8224758,Stat significant at 10%
mu20,-0.0000215884,0.0001875496,-0.1151077,Not Statistically Significant
phi11,0.0394380403,0.0313218660,1.2591217,Not Statistically Significant
phi12,0.1798185232,0.0932765415,1.9278001,Stat significant at 10%
phi21,0.0080611493,0.0103971865,0.7753203,Not Statistically Significant
phi22,0.0824784168,0.0309628296,2.6637881,Stat significant at 1%
a11,0.1991571646,0.0168040309,11.8517495,Stat significant at 1%
a12,0.0039220076,0.0096797205,0.4051778,Not Statistically Significant
a21,-0.0243507771,0.0793781532,-0.3067693,Not Statistically Significant


In [34]:
causality(var_bekk2,cause = "usc")$Granger
causality(var_bekk2,cause = "gbpusd")$Granger


	Granger causality H0: usc do not Granger-cause gbpusd

data:  VAR object var_bekk2
F-Test = 0.60112, df1 = 1, df2 = 2090, p-value = 0.4382



	Granger causality H0: gbpusd do not Granger-cause usc

data:  VAR object var_bekk2
F-Test = 3.7164, df1 = 1, df2 = 2090, p-value = 0.05402


## (c) VAR-BEKK: US Cocoa-CHFUSD

In [35]:
bekk3data <- us_d[,c(1,4)]
head(bekk3data)
tail(bekk3data)

                    usc        chfusd
2020-02-03 -0.016196143 -0.0027972046
2020-02-04  0.009030222 -0.0034833127
2020-02-05  0.012151686 -0.0042739259
2020-02-06  0.017606089 -0.0011687934
2020-02-07  0.011452494 -0.0033679104
2020-02-10 -0.006925235  0.0006354017

                   usc       chfusd
2024-04-26 -0.01962863 -0.002191682
2024-04-29 -0.17075943  0.004059947
2024-04-30  0.03865640 -0.009744124
2024-05-01 -0.11385883  0.004128825
2024-05-02 -0.09105801  0.005524245
2024-05-03  0.07413631  0.005493896

In [36]:
VARselect(bekk3data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.860795e+01,-1.860148e+01,-1.859630e+01,-1.859664e+01,-1.858946e+01,-1.858985e+01,-1.858582e+01,-1.858588e+01,-1.858155e+01,-1.858382e+01
HQ(n),-1.859711e+01,-1.858342e+01,-1.857101e+01,-1.856414e+01,-1.854973e+01,-1.854290e+01,-1.853165e+01,-1.852448e+01,-1.851293e+01,-1.850798e+01
SC(n),-1.857939e+01,-1.855388e+01,-1.852965e+01,-1.851096e+01,-1.848473e+01,-1.846608e+01,-1.844301e+01,-1.842403e+01,-1.840066e+01,-1.838389e+01
FPE(n),8.292216e-09,8.346024e-09,8.389403e-09,8.386498e-09,8.446997e-09,8.443691e-09,8.477769e-09,8.477292e-09,8.514098e-09,8.494788e-09


In [37]:
var_bekk3<-VAR(bekk3data,p = 1,type = "const")
res_bekk3 <- residuals(var_bekk3)
res_bekk3 <- as.data.frame(res_bekk3)

In [38]:
ind_var3 <- index(bekk3data)[-1]
str(ind_var3)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [39]:
res_bekk3 <- xts(res_bekk3,order.by = ind_var3)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk3_fit <- bekk_fit(spec = bekk_spec,data = res_bekk3)

In [40]:
bekk3_fit$A
bekk3_fit$A_t
bekk3_fit$G
bekk3_fit$G_t

0.20079749,0.01767605
0.04667108,0.36211667


10.5682638,1.658173
0.4706513,9.443012


0.97675873,-0.009952466
-0.06977458,0.812802742


138.1039604,-1.77721
-0.8497755,20.68472


### VAR-BEKK Model Table for US Cocoa - CHFUSD

In [41]:
bekk3 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk3)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Estimate"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Estimate"],
                                   summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Estimate"],
                                   bekk3_fit$A[1,1],bekk3_fit$A[1,2],bekk3_fit$A[2,1],bekk3_fit$A[2,2],bekk3_fit$G[1,1],bekk3_fit$G[1,2],bekk3_fit$G[2,1],bekk3_fit$G[2,2]),
                   Std_error = c(summary(var_bekk3)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Std. Error"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Std. Error"],
                                 summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Std. Error"],
                                 bekk3_fit$A_sd[1,1],bekk3_fit$A_sd[1,2],bekk3_fit$A_sd[2,1],bekk3_fit$A_sd[2,2],bekk3_fit$G_sd[1,1],bekk3_fit$G_sd[1,2],bekk3_fit$G_sd[2,1],bekk3_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk3)$varresult$usc$coefficients["const","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["const","t value"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","t value"],
                               summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","t value"],
                               bekk3_fit$A_t[1,1],bekk3_fit$A_t[1,2],bekk3_fit$A_t[2,1],bekk3_fit$A_t[2,2],bekk3_fit$G_t[1,1],bekk3_fit$G_t[1,2],bekk3_fit$G_t[2,1],bekk3_fit$G_t[2,2]))

bekk3$Significant_at_5p_level[abs(bekk3$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.96 & abs(bekk3$T_value) < 2.58] <- "Stat significant at 5%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.645 & abs(bekk3$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) <= 1.645] <- "Not Statistically Significant"
bekk3

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.012669e-03,0.0005659087,1.7894572,Stat significant at 10%
mu20,6.465282e-05,0.0001526395,0.4235655,Not Statistically Significant
phi11,4.662404e-02,0.0311304411,1.4976992,Not Statistically Significant
phi12,6.601048e-02,0.1144900889,0.5765606,Not Statistically Significant
phi21,2.492530e-03,0.0083966454,0.2968483,Not Statistically Significant
phi22,6.931955e-02,0.0308807921,2.2447464,Stat significant at 5%
a11,2.007975e-01,0.0190000455,10.5682638,Stat significant at 1%
a12,1.767605e-02,0.0106599558,1.6581732,Stat significant at 10%
a21,4.667108e-02,0.0991627569,0.4706513,Not Statistically Significant


In [42]:
causality(var_bekk3,cause = "usc")$Granger
causality(var_bekk3,cause = "chfusd")$Granger


	Granger causality H0: usc do not Granger-cause chfusd

data:  VAR object var_bekk3
F-Test = 0.088119, df1 = 1, df2 = 2090, p-value = 0.7666



	Granger causality H0: chfusd do not Granger-cause usc

data:  VAR object var_bekk3
F-Test = 0.33242, df1 = 1, df2 = 2090, p-value = 0.5643


## (d) VAR-BEKK model for US Cocoa - GHSUSD

In [43]:
bekk7data <- na.omit(merge(cr_d[,1],fr_d[,7]))
head(bekk7data)
tail(bekk7data)

                    usc       ghsusd
2020-02-03 -0.016196143  0.009388869
2020-02-04  0.009030222  0.008350745
2020-02-05  0.012151686 -0.003785183
2020-02-06  0.017606089 -0.002762874
2020-02-07  0.011452494  0.002075578
2020-02-10 -0.006925235  0.000589913

                   usc       ghsusd
2024-04-26 -0.01962863 -0.003682821
2024-04-29 -0.17075943 -0.004412793
2024-04-30  0.03865640 -0.001468434
2024-05-01 -0.11385883 -0.002189577
2024-05-02 -0.09105801 -0.003637066
2024-05-03  0.07413631  0.001449549

In [44]:
VARselect(bekk7data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.674036e+01,-1.676221e+01,-1.676807e+01,-1.676751e+01,-1.676154e+01,-1.675665e+01,-1.675276e+01,-1.675037e+01,-1.674375e+01,-1.674268e+01
HQ(n),-1.672952e+01,-1.674415e+01,-1.674279e+01,-1.673501e+01,-1.672181e+01,-1.670970e+01,-1.669859e+01,-1.668897e+01,-1.667513e+01,-1.666683e+01
SC(n),-1.671179e+01,-1.671460e+01,-1.670142e+01,-1.668183e+01,-1.665681e+01,-1.663288e+01,-1.660995e+01,-1.658852e+01,-1.656286e+01,-1.654275e+01
FPE(n),5.367294e-08,5.251300e-08,5.220603e-08,5.223498e-08,5.254804e-08,5.280574e-08,5.301132e-08,5.313856e-08,5.349158e-08,5.354898e-08


In [45]:
var_bekk7<-VAR(bekk7data,p = 3,type = "const")
res_bekk7 <- residuals(var_bekk7)
res_bekk7 <- as.data.frame(res_bekk7)

In [46]:
summary(var_bekk7)


VAR Estimation Results:
Endogenous variables: usc, ghsusd 
Deterministic variables: const 
Sample size: 1046 
Log Likelihood: 5820.953 
Roots of the characteristic polynomial:
0.6277 0.4218 0.4218 0.3185 0.3052 0.3052
Call:
VAR(y = bekk7data, p = 3, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + ghsusd.l1 + usc.l2 + ghsusd.l2 + usc.l3 + ghsusd.l3 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     0.0467707  0.0312800   1.495    0.135
ghsusd.l1 -0.0401381  0.0458489  -0.875    0.382
usc.l2    -0.0084765  0.0316829  -0.268    0.789
ghsusd.l2 -0.0304103  0.0455180  -0.668    0.504
usc.l3     0.0308992  0.0322329   0.959    0.338
ghsusd.l3  0.0232327  0.0458462   0.507    0.612
const      0.0009274  0.0005726   1.620    0.106


Residual standard error: 0.01832 on 1039 degrees of freedom
Multiple R-Squared: 0.004477,	Adjusted R-squared: -0.001272 
F-statistic: 0.7787 on 6 and 1039 DF,  p-value: 0.5867 


Estimation results for equation ghsusd: 
gh

In [47]:
ind_var7 <- index(bekk7data)[-c(1:3)]
str(ind_var7)

 Date[1:1046], format: "2020-02-06" "2020-02-07" "2020-02-10" "2020-02-11" "2020-02-12" ...


In [48]:
res_bekk7 <- xts(res_bekk7,order.by = ind_var7)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk7_fit <- bekk_fit(spec = bekk_spec,data = res_bekk7)

In [49]:
bekk7_fit$A
bekk7_fit$A_t
bekk7_fit$G
bekk7_fit$G_t

0.15046169,0.006030409
0.03441847,0.397010034


8.177057,0.7703507
1.274523,25.6869749


0.91588739,-0.004879871
0.03214397,0.918783246


39.30338,-0.3862518
2.53961,120.8713615


### VAR-BEKK Model Table for US Cocoa - GHSUSD

In [50]:
bekk7 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi21","phi22","phi23","phi24","phi25","phi26","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk7)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Estimate"],
                                   bekk7_fit$A[1,1],bekk7_fit$A[1,2],bekk7_fit$A[2,1],bekk7_fit$A[2,2],bekk7_fit$G[1,1],bekk7_fit$G[1,2],bekk7_fit$G[2,1],bekk7_fit$G[2,2]),
                   Std_error = c(summary(var_bekk7)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Std. Error"],
                                   bekk7_fit$A_sd[1,1],bekk7_fit$A_sd[1,2],bekk7_fit$A_sd[2,1],bekk7_fit$A_sd[2,2],bekk7_fit$G_sd[1,1],bekk7_fit$G_sd[1,2],bekk7_fit$G_sd[2,1],bekk7_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk7)$varresult$usc$coefficients["const","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","t value"],
                                   bekk7_fit$A_t[1,1],bekk7_fit$A_t[1,2],bekk7_fit$A_t[2,1],bekk7_fit$A_t[2,2],bekk7_fit$G_t[1,1],bekk7_fit$G_t[1,2],bekk7_fit$G_t[2,1],bekk7_fit$G_t[2,2]))

bekk7$Significant_at_5p_level[abs(bekk7$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.96 & abs(bekk7$T_value) < 2.58] <- "Stat significant at 5%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.645 & abs(bekk7$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) <= 1.645] <- "Not Statistically Significant"
bekk7

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0009274335,0.0005725617,1.6197966,Not Statistically Significant
mu20,-0.0006062670,0.0003850192,-1.5746411,Not Statistically Significant
phi11,0.0467706681,0.0312800482,1.4952237,Not Statistically Significant
phi12,-0.0401381215,0.0458489301,-0.8754429,Not Statistically Significant
phi13,-0.0084765177,0.0316828976,-0.2675424,Not Statistically Significant
phi14,-0.0304103067,0.0455180031,-0.6680940,Not Statistically Significant
phi15,0.0308992290,0.0322328753,0.9586247,Not Statistically Significant
phi16,0.0232326557,0.0458462250,0.5067518,Not Statistically Significant
phi21,0.0103161699,0.0210342723,0.4904458,Not Statistically Significant


In [51]:
causality(var_bekk7,cause = "usc")$Granger
causality(var_bekk7,cause = "ghsusd")$Granger


	Granger causality H0: usc do not Granger-cause ghsusd

data:  VAR object var_bekk7
F-Test = 0.18512, df1 = 3, df2 = 2078, p-value = 0.9066



	Granger causality H0: ghsusd do not Granger-cause usc

data:  VAR object var_bekk7
F-Test = 0.47915, df1 = 3, df2 = 2078, p-value = 0.6968


## (iii) VAR-BEKK Models for London Cocoa

In [52]:
l_d <- na.omit(merge(cr_d[,2],fr_d[,c(4:6)]))
str(l_d)
head(l_d)
tail(l_d)

An xts object on 2020-02-03 / 2024-05-03 containing: 
  Data:    double [1049, 4]
  Columns: lc, usdgbp, eurgbp, chfgbp
  Index:   Date [1049] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3342] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3342] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                     lc       usdgbp       eurgbp        chfgbp
2020-02-03 -0.005089070  0.015717416  0.012889672  0.0129490487
2020-02-04  0.000000000 -0.002732775 -0.004120804 -0.0062956642
2020-02-05  0.019202207  0.002212822 -0.002007677 -0.0020230124
2020-02-06  0.015889113  0.005575379  0.004129069  0.0043570302
2020-02-07  0.007361996  0.002905008  0.000000000 -0.0004411533
2020-02-10 -0.002447982 -0.001741992 -0.005311941 -0.0010721833

                    lc        usdgbp        eurgbp        chfgbp
2024-04-26 -0.02951879  0.0018128971 -0.0015173624 -0.0003996004
2024-04-29 -0.15652942 -0.0056998056 -0.0031588210 -0.0016571906
2024-04-30  0.03216783  0.0056998056  0.0004686036 -0.0040688901
2024-05-01 -0.08952546 -0.0028145248  0.0018722215  0.0013772526
2024-05-02 -0.10077335 -0.0006265272  0.0001168975  0.0049196369
2024-05-03  0.04940746 -0.0009405274  0.0025682947  0.0044411620

## (a) VAR-BEKK: London-USDGBP

In [53]:
bekk4data <- l_d[,c(1,2)]
head(bekk4data)
tail(bekk4data)

                     lc       usdgbp
2020-02-03 -0.005089070  0.015717416
2020-02-04  0.000000000 -0.002732775
2020-02-05  0.019202207  0.002212822
2020-02-06  0.015889113  0.005575379
2020-02-07  0.007361996  0.002905008
2020-02-10 -0.002447982 -0.001741992

                    lc        usdgbp
2024-04-26 -0.02951879  0.0018128971
2024-04-29 -0.15652942 -0.0056998056
2024-04-30  0.03216783  0.0056998056
2024-05-01 -0.08952546 -0.0028145248
2024-05-02 -0.10077335 -0.0006265272
2024-05-03  0.04940746 -0.0009405274

In [54]:
VARselect(bekk4data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.843614e+01,-1.842888e+01,-1.842457e+01,-1.842764e+01,-1.842448e+01,-1.842850e+01,-1.842345e+01,-1.841672e+01,-1.841391e+01,-1.841107e+01
HQ(n),-1.842530e+01,-1.841082e+01,-1.839929e+01,-1.839514e+01,-1.838475e+01,-1.838155e+01,-1.836928e+01,-1.835532e+01,-1.834528e+01,-1.833522e+01
SC(n),-1.840757e+01,-1.838128e+01,-1.835793e+01,-1.834196e+01,-1.831975e+01,-1.830473e+01,-1.828064e+01,-1.825487e+01,-1.823301e+01,-1.821113e+01
FPE(n),9.846646e-09,9.918352e-09,9.961154e-09,9.930619e-09,9.962133e-09,9.922096e-09,9.972342e-09,1.003975e-08,1.006806e-08,1.009669e-08


In [55]:
var_bekk4<-VAR(bekk4data,p = 1,type = "const")
res_bekk4 <- residuals(var_bekk4)
res_bekk4 <- as.data.frame(res_bekk4)

In [56]:
summary(var_bekk4)


VAR Estimation Results:
Endogenous variables: lc, usdgbp 
Deterministic variables: const 
Sample size: 1048 
Log Likelihood: 6698.532 
Roots of the characteristic polynomial:
0.1019 0.1019
Call:
VAR(y = bekk4data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + usdgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)   
lc.l1     0.1013236  0.0310904   3.259  0.00115 **
usdgbp.l1 0.0573152  0.0835905   0.686  0.49308   
const     0.0010140  0.0005069   2.000  0.04572 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01637 on 1045 degrees of freedom
Multiple R-Squared: 0.01112,	Adjusted R-squared: 0.00923 
F-statistic: 5.877 on 2 and 1045 DF,  p-value: 0.002896 


Estimation results for equation usdgbp: 
usdgbp = lc.l1 + usdgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)   
lc.l1     -1.925e-02  1.149e-02  -1.676  0.09405 . 
usdgbp.l1  9.153e-02  3.088e-02   2.964  0.00311 **
const 

In [57]:
ind_var4 <- index(bekk4data)[-1]
str(ind_var4)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [58]:
res_bekk4 <- xts(res_bekk4,order.by = ind_var4)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk4_fit <- bekk_fit(spec = bekk_spec,data = res_bekk4)

In [59]:
bekk4_fit$A
bekk4_fit$A_t
bekk4_fit$G
bekk4_fit$G_t

0.22468327,-0.004161311
0.07440445,0.282508812


13.155437,-0.4087771
1.337532,12.6719040


0.97336166,0.002487647
-0.02082044,0.939264696


172.615305,0.7203405
-1.000403,88.7277750


### VAR-BEKK Model Table for London Cocoa - USDGBP

In [60]:
bekk4 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk4)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Estimate"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Estimate"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Estimate"],
                                   bekk4_fit$A[1,1],bekk4_fit$A[1,2],bekk4_fit$A[2,1],bekk4_fit$A[2,2],bekk4_fit$G[1,1],bekk4_fit$G[1,2],bekk4_fit$G[2,1],bekk4_fit$G[2,2]),
                   Std_error = c(summary(var_bekk4)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Std. Error"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Std. Error"],
                                   bekk4_fit$A_sd[1,1],bekk4_fit$A_sd[1,2],bekk4_fit$A_sd[2,1],bekk4_fit$A_sd[2,2],bekk4_fit$G_sd[1,1],bekk4_fit$G_sd[1,2],bekk4_fit$G_sd[2,1],bekk4_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk4)$varresult$lc$coefficients["const","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","t value"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","t value"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","t value"],
                                   bekk4_fit$A_t[1,1],bekk4_fit$A_t[1,2],bekk4_fit$A_t[2,1],bekk4_fit$A_t[2,2],bekk4_fit$G_t[1,1],bekk4_fit$G_t[1,2],bekk4_fit$G_t[2,1],bekk4_fit$G_t[2,2]))

bekk4$Significant_at_5p_level[abs(bekk4$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.96 & abs(bekk4$T_value) < 2.58] <- "Stat significant at 5%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.645 & abs(bekk4$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) <= 1.645] <- "Not Statistically Significant"
bekk4

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.013980e-03,0.0005069100,2.0003149,Stat significant at 5%
mu20,3.495419e-05,0.0001872824,0.1866390,Not Statistically Significant
phi11,1.013236e-01,0.0310903604,3.2590036,Stat significant at 1%
phi12,5.731523e-02,0.0835905471,0.6856665,Not Statistically Significant
phi21,-1.925055e-02,0.0114866062,-1.6759131,Stat significant at 10%
phi22,9.152960e-02,0.0308832603,2.9637286,Stat significant at 1%
a11,2.246833e-01,0.0170791196,13.1554366,Stat significant at 1%
a12,-4.161311e-03,0.0101799016,-0.4087771,Not Statistically Significant
a21,7.440445e-02,0.0556281706,1.3375318,Not Statistically Significant


In [61]:
causality(var_bekk4,cause = "lc")$Granger
causality(var_bekk4,cause = "usdgbp")$Granger


	Granger causality H0: lc do not Granger-cause usdgbp

data:  VAR object var_bekk4
F-Test = 2.8087, df1 = 1, df2 = 2090, p-value = 0.0939



	Granger causality H0: usdgbp do not Granger-cause lc

data:  VAR object var_bekk4
F-Test = 0.47014, df1 = 1, df2 = 2090, p-value = 0.493


## (b) VAR-BEKK: London Cocoa-EURGBP

In [62]:
bekk5data <- l_d[,c(1,3)]
head(bekk5data)

                     lc       eurgbp
2020-02-03 -0.005089070  0.012889672
2020-02-04  0.000000000 -0.004120804
2020-02-05  0.019202207 -0.002007677
2020-02-06  0.015889113  0.004129069
2020-02-07  0.007361996  0.000000000
2020-02-10 -0.002447982 -0.005311941

In [63]:
VARselect(bekk5data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.915257e+01,-1.914770e+01,-1.914531e+01,-1.915038e+01,-1.914921e+01,-1.914521e+01,-1.914097e+01,-1.913538e+01,-1.913148e+01,-1.912508e+01
HQ(n),-1.914173e+01,-1.912964e+01,-1.912003e+01,-1.911787e+01,-1.910948e+01,-1.909826e+01,-1.908679e+01,-1.907398e+01,-1.906286e+01,-1.904923e+01
SC(n),-1.912401e+01,-1.910009e+01,-1.907867e+01,-1.906469e+01,-1.904449e+01,-1.902144e+01,-1.899816e+01,-1.897353e+01,-1.895059e+01,-1.892515e+01
FPE(n),4.809996e-09,4.833498e-09,4.845039e-09,4.820555e-09,4.826168e-09,4.845528e-09,4.866130e-09,4.893407e-09,4.912556e-09,4.944104e-09


In [64]:
var_bekk5<-VAR(bekk5data,p = 1,type = "const")
res_bekk5 <- residuals(var_bekk5)
res_bekk5 <- as.data.frame(res_bekk5)

In [65]:
summary(var_bekk5)


VAR Estimation Results:
Endogenous variables: lc, eurgbp 
Deterministic variables: const 
Sample size: 1048 
Log Likelihood: 7070.065 
Roots of the characteristic polynomial:
0.1033 0.009415
Call:
VAR(y = bekk5data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + eurgbp.l1 + const 

          Estimate Std. Error t value Pr(>|t|)    
lc.l1     0.103715   0.030958   3.350 0.000837 ***
eurgbp.l1 0.002473   0.119457   0.021 0.983486    
const     0.001013   0.000507   1.999 0.045919 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01638 on 1045 degrees of freedom
Multiple R-Squared: 0.01068,	Adjusted R-squared: 0.008785 
F-statistic:  5.64 on 2 and 1045 DF,  p-value: 0.003663 


Estimation results for equation eurgbp: 
eurgbp = lc.l1 + eurgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
lc.l1     -1.514e-02  7.986e-03  -1.895   0.0583 .
eurgbp.l1  9.018e-03  3.081e-02   0.293   0.7698  
const 

In [66]:
ind_var5 <- index(bekk5data)[-1]
str(ind_var5)

 Date[1:1048], format: "2020-02-04" "2020-02-05" "2020-02-06" "2020-02-07" "2020-02-10" ...


In [67]:
res_bekk5 <- xts(res_bekk5,order.by = ind_var5)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk5_fit <- bekk_fit(spec = bekk_spec,data = res_bekk5)

In [68]:
bekk5_fit$A
bekk5_fit$A_t
bekk5_fit$G
bekk5_fit$G_t

0.29906394,-0.004960049
0.03741119,0.193709073


12.6056768,-1.139831
0.5522428,13.223307


0.945226175,0.002296879
-0.003461547,0.978856263


90.1543000,1.907254
-0.1819765,290.256557


### VAR-BEKK Model Table for London-EURGBP

In [69]:
bekk5 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk5)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Estimate"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Estimate"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Estimate"],
                                   bekk5_fit$A[1,1],bekk5_fit$A[1,2],bekk5_fit$A[2,1],bekk5_fit$A[2,2],bekk5_fit$G[1,1],bekk5_fit$G[1,2],bekk5_fit$G[2,1],bekk5_fit$G[2,2]),
                   Std_error = c(summary(var_bekk5)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Std. Error"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Std. Error"],
                                   bekk5_fit$A_sd[1,1],bekk5_fit$A_sd[1,2],bekk5_fit$A_sd[2,1],bekk5_fit$A_sd[2,2],bekk5_fit$G_sd[1,1],bekk5_fit$G_sd[1,2],bekk5_fit$G_sd[2,1],bekk5_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk5)$varresult$lc$coefficients["const","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","t value"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","t value"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","t value"],
                                   bekk5_fit$A_t[1,1],bekk5_fit$A_t[1,2],bekk5_fit$A_t[2,1],bekk5_fit$A_t[2,2],bekk5_fit$G_t[1,1],bekk5_fit$G_t[1,2],bekk5_fit$G_t[2,1],bekk5_fit$G_t[2,2]))

bekk5$Significant_at_5p_level[abs(bekk5$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.96 & abs(bekk5$T_value) < 2.58] <- "Stat significant at 5%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.645 & abs(bekk5$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) <= 1.645] <- "Not Statistically Significant"
bekk5

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.013298e-03,0.0005070241,1.99852091,Stat significant at 5%
mu20,1.519101e-05,0.0001307880,0.11614993,Not Statistically Significant
phi11,1.037151e-01,0.0309584349,3.35013943,Stat significant at 1%
phi12,2.473222e-03,0.1194569348,0.02070388,Not Statistically Significant
phi21,-1.513695e-02,0.0079857952,-1.89548425,Stat significant at 10%
phi22,9.017843e-03,0.0308141745,0.29265242,Not Statistically Significant
a11,2.990639e-01,0.0237245444,12.60567679,Stat significant at 1%
a12,-4.960049e-03,0.0043515661,-1.13983066,Not Statistically Significant
a21,3.741119e-02,0.0677440980,0.55224284,Not Statistically Significant


In [70]:
causality(var_bekk5,cause = "lc")$Granger
causality(var_bekk5,cause = "eurgbp")$Granger


	Granger causality H0: lc do not Granger-cause eurgbp

data:  VAR object var_bekk5
F-Test = 3.5929, df1 = 1, df2 = 2090, p-value = 0.05817



	Granger causality H0: eurgbp do not Granger-cause lc

data:  VAR object var_bekk5
F-Test = 0.00042865, df1 = 1, df2 = 2090, p-value = 0.9835


## (c) VAR-BEKK: London-CHFGBP

In [71]:
bekk6data <- l_d[,c(1,4)]
head(bekk6data)
tail(bekk6data)

                     lc        chfgbp
2020-02-03 -0.005089070  0.0129490487
2020-02-04  0.000000000 -0.0062956642
2020-02-05  0.019202207 -0.0020230124
2020-02-06  0.015889113  0.0043570302
2020-02-07  0.007361996 -0.0004411533
2020-02-10 -0.002447982 -0.0010721833

                    lc        chfgbp
2024-04-26 -0.02951879 -0.0003996004
2024-04-29 -0.15652942 -0.0016571906
2024-04-30  0.03216783 -0.0040688901
2024-05-01 -0.08952546  0.0013772526
2024-05-02 -0.10077335  0.0049196369
2024-05-03  0.04940746  0.0044411620

In [72]:
VARselect(bekk6data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.880232e+01,-1.879657e+01,-1.879627e+01,-1.880241e+01,-1.879846e+01,-1.879639e+01,-1.879069e+01,-1.878565e+01,-1.878404e+01,-1.878057e+01
HQ(n),-1.879148e+01,-1.877852e+01,-1.877099e+01,-1.876990e+01,-1.875873e+01,-1.874943e+01,-1.873651e+01,-1.872425e+01,-1.871542e+01,-1.870473e+01
SC(n),-1.877376e+01,-1.874897e+01,-1.872963e+01,-1.871672e+01,-1.869373e+01,-1.867262e+01,-1.864788e+01,-1.862380e+01,-1.860315e+01,-1.858064e+01
FPE(n),6.827427e-09,6.866757e-09,6.868843e-09,6.826800e-09,6.853854e-09,6.868055e-09,6.907304e-09,6.942208e-09,6.953427e-09,6.977590e-09


In [73]:
var_bekk6<-VAR(bekk6data,p = 4,type = "const")
res_bekk6 <- residuals(var_bekk6)
res_bekk6 <- as.data.frame(res_bekk6)

In [74]:
summary(var_bekk6)


VAR Estimation Results:
Endogenous variables: lc, chfgbp 
Deterministic variables: const 
Sample size: 1045 
Log Likelihood: 6880.502 
Roots of the characteristic polynomial:
0.5785 0.5459 0.5459 0.5032 0.4784 0.4784 0.2883 0.2883
Call:
VAR(y = bekk6data, p = 4, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + chfgbp.l1 + lc.l2 + chfgbp.l2 + lc.l3 + chfgbp.l3 + lc.l4 + chfgbp.l4 + const 

            Estimate Std. Error t value Pr(>|t|)   
lc.l1      0.1007954  0.0311369   3.237  0.00125 **
chfgbp.l1 -0.0174823  0.1013595  -0.172  0.86310   
lc.l2     -0.0114039  0.0318755  -0.358  0.72059   
chfgbp.l2 -0.1015459  0.1011236  -1.004  0.31553   
lc.l3      0.0305516  0.0322695   0.947  0.34398   
chfgbp.l3  0.0791625  0.1011078   0.783  0.43383   
lc.l4      0.0926807  0.0322501   2.874  0.00414 **
chfgbp.l4  0.0395760  0.1008981   0.392  0.69496   
const      0.0008487  0.0005106   1.662  0.09678 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [75]:
ind_var6 <- index(bekk6data)[-c(1:4)]
str(ind_var6)

 Date[1:1045], format: "2020-02-07" "2020-02-10" "2020-02-11" "2020-02-12" "2020-02-13" ...


In [76]:
res_bekk6 <- xts(res_bekk6,order.by = ind_var6)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk6_fit <- bekk_fit(spec = bekk_spec,data = res_bekk6)

In [77]:
bekk6_fit$A
bekk6_fit$A_t
bekk6_fit$G
bekk6_fit$G_t

0.23772798,-0.006151465
0.06182434,0.226850762


11.5843029,-0.7881537
0.9577618,13.7282100


0.96976549,0.003340487
-0.01993112,0.961299205


137.086980,1.162797
-0.925166,147.991759


### VAR-BEKK Model Table for London-CHFGBP

In [78]:
bekk6 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk6)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l2","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l2","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l3","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l3","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l4","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l4","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l2","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l2","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l3","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l3","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l4","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l4","Estimate"],
                                   bekk6_fit$A[1,1],bekk6_fit$A[1,2],bekk6_fit$A[2,1],bekk6_fit$A[2,2],bekk6_fit$G[1,1],bekk6_fit$G[1,2],bekk6_fit$G[2,1],bekk6_fit$G[2,2]),
                   Std_error = c(summary(var_bekk6)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l2","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l2","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l3","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l3","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l4","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l4","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l2","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l2","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l3","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l3","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l4","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l4","Std. Error"],
                                   bekk6_fit$A_sd[1,1],bekk6_fit$A_sd[1,2],bekk6_fit$A_sd[2,1],bekk6_fit$A_sd[2,2],bekk6_fit$G_sd[1,1],bekk6_fit$G_sd[1,2],bekk6_fit$G_sd[2,1],bekk6_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk6)$varresult$lc$coefficients["const","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l2","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l2","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l3","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l3","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l4","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l4","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l2","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l2","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l3","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l3","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l4","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l4","t value"],
                                   bekk6_fit$A_t[1,1],bekk6_fit$A_t[1,2],bekk6_fit$A_t[2,1],bekk6_fit$A_t[2,2],bekk6_fit$G_t[1,1],bekk6_fit$G_t[1,2],bekk6_fit$G_t[2,1],bekk6_fit$G_t[2,2]))

bekk6$Significant_at_5p_level[abs(bekk6$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.96 & abs(bekk6$T_value) < 2.58] <- "Stat significant at 5%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.645 & abs(bekk6$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) <= 1.645] <- "Not Statistically Significant"
bekk6

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0008486502,0.0005105599,1.6621951,Stat significant at 10%
mu20,0.0001521000,0.0001560854,0.9744665,Not Statistically Significant
phi11,0.1007954464,0.0311369415,3.2371659,Stat significant at 1%
phi12,-0.0174823344,0.1013595138,-0.1724785,Not Statistically Significant
phi13,-0.0114038792,0.0318754566,-0.3577636,Not Statistically Significant
phi14,-0.1015459238,0.1011235938,-1.0041764,Not Statistically Significant
phi15,0.0305516218,0.0322694738,0.9467654,Not Statistically Significant
phi16,0.0791624625,0.1011077730,0.7829513,Not Statistically Significant
phi17,0.0926806800,0.0322500932,2.8738112,Stat significant at 1%


In [79]:
causality(var_bekk6,cause = "lc")$Granger
causality(var_bekk6,cause = "chfgbp")$Granger


	Granger causality H0: lc do not Granger-cause chfgbp

data:  VAR object var_bekk6
F-Test = 2.6352, df1 = 4, df2 = 2072, p-value = 0.03253



	Granger causality H0: chfgbp do not Granger-cause lc

data:  VAR object var_bekk6
F-Test = 0.45244, df1 = 4, df2 = 2072, p-value = 0.7707


## (d) VAR-BEKK Model for London Cocoa - GHSGBP

In [80]:
bekk8data <- na.omit(merge(cr_d[,2],fr_d[,8]))
head(bekk8data)
tail(bekk8data)

                     lc       ghsgbp
2020-02-03 -0.005089070  0.025123068
2020-02-04  0.000000000  0.005584836
2020-02-05  0.019202207 -0.001560331
2020-02-06  0.015889113  0.002799130
2020-02-07  0.007361996  0.004971314
2020-02-10 -0.002447982 -0.001110366

                    lc        ghsgbp
2024-04-26 -0.02951879 -0.0018400829
2024-04-29 -0.15652942 -0.0101599929
2024-04-30  0.03216783  0.0042437477
2024-05-01 -0.08952546 -0.0050328783
2024-05-02 -0.10077335 -0.0041962674
2024-05-03  0.04940746  0.0004565954

In [81]:
VARselect(bekk8data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.674173e+01,-1.676176e+01,-1.676749e+01,-1.677628e+01,-1.677246e+01,-1.677225e+01,-1.676726e+01,-1.676289e+01,-1.675728e+01,-1.675403e+01
HQ(n),-1.673090e+01,-1.674370e+01,-1.674221e+01,-1.674378e+01,-1.673273e+01,-1.672529e+01,-1.671308e+01,-1.670149e+01,-1.668865e+01,-1.667819e+01
SC(n),-1.671317e+01,-1.671415e+01,-1.670085e+01,-1.669060e+01,-1.666773e+01,-1.664848e+01,-1.662445e+01,-1.660104e+01,-1.657638e+01,-1.655410e+01
FPE(n),5.359915e-08,5.253654e-08,5.223616e-08,5.177885e-08,5.197743e-08,5.198852e-08,5.224841e-08,5.247710e-08,5.277288e-08,5.294445e-08


In [82]:
var_bekk8<-VAR(bekk8data,p = 4,type = "const")
res_bekk8 <- residuals(var_bekk8)
res_bekk8 <- as.data.frame(res_bekk8)

In [83]:
summary(var_bekk8)


VAR Estimation Results:
Endogenous variables: lc, ghsgbp 
Deterministic variables: const 
Sample size: 1045 
Log Likelihood: 5822.864 
Roots of the characteristic polynomial:
0.5667 0.5667 0.5431 0.5431 0.5416 0.5416 0.505 0.286
Call:
VAR(y = bekk8data, p = 4, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + ghsgbp.l1 + lc.l2 + ghsgbp.l2 + lc.l3 + ghsgbp.l3 + lc.l4 + ghsgbp.l4 + const 

            Estimate Std. Error t value Pr(>|t|)   
lc.l1      0.1000931  0.0310762   3.221  0.00132 **
ghsgbp.l1 -0.0144869  0.0367237  -0.394  0.69330   
lc.l2     -0.0101741  0.0318254  -0.320  0.74927   
ghsgbp.l2 -0.0520419  0.0366424  -1.420  0.15583   
lc.l3      0.0293684  0.0322470   0.911  0.36265   
ghsgbp.l3  0.0290613  0.0366335   0.793  0.42779   
lc.l4      0.0936026  0.0321913   2.908  0.00372 **
ghsgbp.l4  0.0737984  0.0365911   2.017  0.04397 * 
const      0.0008778  0.0005116   1.716  0.08650 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


R

In [84]:
ind_var8 <- index(bekk8data)[-c(1:4)]
str(ind_var8)

 Date[1:1045], format: "2020-02-07" "2020-02-10" "2020-02-11" "2020-02-12" "2020-02-13" ...


In [85]:
res_bekk8 <- xts(res_bekk8,order.by = ind_var8)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk8_fit <- bekk_fit(spec = bekk_spec,data = res_bekk8)

In [86]:
bekk8_fit$A
bekk8_fit$A_t
bekk8_fit$G
bekk8_fit$G_t

0.190126213,-0.01399911
0.006686038,0.54046104


12.6954898,-1.042015
0.4191365,22.134929


0.980505793,0.007636664
-0.002062382,0.841371429


211.8239750,1.038077
-0.2126507,72.375853


### VAR-BEKK Model Table for London Cocoa - GHSGBP

In [87]:
bekk8 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk8)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","Estimate"],
                                   bekk8_fit$A[1,1],bekk8_fit$A[1,2],bekk8_fit$A[2,1],bekk8_fit$A[2,2],bekk8_fit$G[1,1],bekk8_fit$G[1,2],bekk8_fit$G[2,1],bekk8_fit$G[2,2]),
                   Std_error = c(summary(var_bekk8)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","Std. Error"],
                                   bekk8_fit$A_sd[1,1],bekk8_fit$A_sd[1,2],bekk8_fit$A_sd[2,1],bekk8_fit$A_sd[2,2],bekk8_fit$G_sd[1,1],bekk8_fit$G_sd[1,2],bekk8_fit$G_sd[2,1],bekk8_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk8)$varresult$lc$coefficients["const","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","t value"],
                                   bekk8_fit$A_t[1,1],bekk8_fit$A_t[1,2],bekk8_fit$A_t[2,1],bekk8_fit$A_t[2,2],bekk8_fit$G_t[1,1],bekk8_fit$G_t[1,2],bekk8_fit$G_t[2,1],bekk8_fit$G_t[2,2]))

bekk8$Significant_at_5p_level[abs(bekk8$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.96 & abs(bekk8$T_value) < 2.58] <- "Stat significant at 5%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.645 & abs(bekk8$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) <= 1.645] <- "Not Statistically Significant"
bekk8

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0008777824,0.0005115890,1.7157959,Stat significant at 10%
mu20,-0.0005616539,0.0004322514,-1.2993687,Not Statistically Significant
phi11,0.1000931401,0.0310762436,3.2208893,Stat significant at 1%
phi12,-0.0144869442,0.0367236960,-0.3944849,Not Statistically Significant
phi13,-0.0101741202,0.0318253502,-0.3196860,Not Statistically Significant
phi14,-0.0520419046,0.0366424478,-1.4202628,Not Statistically Significant
phi15,0.0293683519,0.0322469851,0.9107317,Not Statistically Significant
phi16,0.0290613138,0.0366335487,0.7932978,Not Statistically Significant
phi17,0.0936025694,0.0321913280,2.9076952,Stat significant at 1%


In [88]:
causality(var_bekk8,cause = "lc")$Granger
causality(var_bekk8,cause = "ghsgbp")$Granger


	Granger causality H0: lc do not Granger-cause ghsgbp

data:  VAR object var_bekk8
F-Test = 1.2699, df1 = 4, df2 = 2072, p-value = 0.2796



	Granger causality H0: ghsgbp do not Granger-cause lc

data:  VAR object var_bekk8
F-Test = 1.5255, df1 = 4, df2 = 2072, p-value = 0.1921


# Summary of Period 4 models

In [89]:
bekk_c <- rep(NA,14)
for(i in 1:14){
    bekk_c[i] = paste0(round(bekkc[i,2],4)," (",round(bekkc[i,3],4),")")
}

bekk_1 <- rep(NA,14)
for(i in 1:14){
    bekk_1[i] = paste0(round(bekk1[i,2],4)," (",round(bekk1[i,3],4),")")
}

bekk_2 <- rep(NA,14)
for(i in 1:14){
    bekk_2[i] = paste0(round(bekk2[i,2],4)," (",round(bekk2[i,3],4),")")
}

bekk_3 <- rep(NA,14)
for(i in 1:14){
    bekk_3[i] = paste0(round(bekk3[i,2],4)," (",round(bekk3[i,3],4),")")
}

bekk_4 <- rep(NA,14)
for(i in 1:14){
    bekk_4[i] = paste0(round(bekk4[i,2],4)," (",round(bekk4[i,3],4),")")
}

bekk_5 <- rep(NA,14)
for(i in 1:14){
    bekk_5[i] = paste0(round(bekk5[i,2],4)," (",round(bekk5[i,3],4),")")
}

bekk_6 <- rep(NA,26)
for(i in 1:26){
    bekk_6[i] = paste0(round(bekk6[i,2],4)," (",round(bekk6[i,3],4),")")
}

bekk_7 <- rep(NA,22)
for(i in 1:22){
    bekk_7[i] = paste0(round(bekk7[i,2],4)," (",round(bekk7[i,3],4),")")
}


bekk_8 <- rep(NA,26)
for(i in 1:26){
    bekk_8[i] = paste0(round(bekk8[i,2],4)," (",round(bekk8[i,3],4),")")
}

In [90]:
parameters1 = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22")
parameters2 = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi21","phi22","phi23","phi24","phi25","phi26","a11","a12","a21","a22","g11","g12","g21","g22")
parameters3 = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22")

In [91]:
daily_bekk1 <- data.frame("Parameters" = parameters1,"BEKK1" = bekk_1,"BEKK3" = bekk_3,"BEKK4" = bekk_4,"BEKK5" = bekk_5)
daily_bekk2 <- data.frame("Parameters" = parameters2,"BEKK7" = bekk_7)
daily_bekk3 <- data.frame("Parameters" = parameters3,"BEKK6" = bekk_6,"BEKK8" = bekk_8)

In [92]:
daily_bekk1

Parameters,BEKK1,BEKK3,BEKK4,BEKK5
<chr>,<chr>,<chr>,<chr>,<chr>
mu10,0.001 (6e-04),0.001 (6e-04),0.001 (5e-04),0.001 (5e-04)
mu20,0 (2e-04),1e-04 (2e-04),0 (2e-04),0 (1e-04)
phi11,0.0438 (0.0312),0.0466 (0.0311),0.1013 (0.0311),0.1037 (0.031)
phi12,0.1342 (0.1157),0.066 (0.1145),0.0573 (0.0836),0.0025 (0.1195)
phi21,0.001 (0.0084),0.0025 (0.0084),-0.0193 (0.0115),-0.0151 (0.008)
phi22,0.0342 (0.031),0.0693 (0.0309),0.0915 (0.0309),0.009 (0.0308)
a11,0.2178 (0.0207),0.2008 (0.019),0.2247 (0.0171),0.2991 (0.0237)
a12,0.0029 (0.0066),0.0177 (0.0107),-0.0042 (0.0102),-0.005 (0.0044)
a21,-0.1101 (0.0876),0.0467 (0.0992),0.0744 (0.0556),0.0374 (0.0677)


In [93]:
daily_bekk2

Parameters,BEKK7
<chr>,<chr>
mu10,9e-04 (6e-04)
mu20,-6e-04 (4e-04)
phi11,0.0468 (0.0313)
phi12,-0.0401 (0.0458)
phi13,-0.0085 (0.0317)
phi14,-0.0304 (0.0455)
phi15,0.0309 (0.0322)
phi16,0.0232 (0.0458)
phi21,0.0103 (0.021)


In [94]:
daily_bekk3

Parameters,BEKK6,BEKK8
<chr>,<chr>,<chr>
mu10,8e-04 (5e-04),9e-04 (5e-04)
mu20,2e-04 (2e-04),-6e-04 (4e-04)
phi11,0.1008 (0.0311),0.1001 (0.0311)
phi12,-0.0175 (0.1014),-0.0145 (0.0367)
phi13,-0.0114 (0.0319),-0.0102 (0.0318)
phi14,-0.1015 (0.1011),-0.052 (0.0366)
phi15,0.0306 (0.0323),0.0294 (0.0322)
phi16,0.0792 (0.1011),0.0291 (0.0366)
phi17,0.0927 (0.0323),0.0936 (0.0322)


# Final VAR-BEKK Table for Period 2 (Table A.5)

In [95]:
daily_bekk <- data.frame("Parameters" = parameters3,
                         "BEKKC"=c(bekk_c[1:4],rep(NA,6),bekk_c[5:6],rep(NA,6),bekk_c[7:14]),
                         "BEKK1"=c(bekk_1[1:4],rep(NA,6),bekk_1[5:6],rep(NA,6),bekk_1[7:14]),
                         "BEKK2"=c(bekk_2[1:4],rep(NA,6),bekk_2[5:6],rep(NA,6),bekk_2[7:14]),
                          "BEKK3"=c(bekk_3[1:4],rep(NA,6),bekk_3[5:6],rep(NA,6),bekk_3[7:14]),
                          "BEKK4"=c(bekk_4[1:4],rep(NA,6),bekk_4[5:6],rep(NA,6),bekk_4[7:14]),
                         "BEKK5"=c(bekk_5[1:4],rep(NA,6),bekk_5[5:6],rep(NA,6),bekk_5[7:14]),
                          "BEKK6"=bekk_6,
                         "BEKK7" = c(bekk_7[1:8],rep(NA,2),bekk_7[9:14],rep(NA,2),bekk_7[15:22]),
                         "BEKK8" = bekk_8)
daily_bekk

Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,0.001 (6e-04),0.001 (6e-04),0.001 (6e-04),0.001 (6e-04),0.001 (5e-04),0.001 (5e-04),8e-04 (5e-04),9e-04 (6e-04),9e-04 (5e-04)
mu20,0.001 (5e-04),0 (2e-04),0 (2e-04),1e-04 (2e-04),0 (2e-04),0 (1e-04),2e-04 (2e-04),-6e-04 (4e-04),-6e-04 (4e-04)
phi11,0.0319 (0.0673),0.0438 (0.0312),0.0394 (0.0313),0.0466 (0.0311),0.1013 (0.0311),0.1037 (0.031),0.1008 (0.0311),0.0468 (0.0313),0.1001 (0.0311)
phi12,0.019 (0.0746),0.1342 (0.1157),0.1798 (0.0933),0.066 (0.1145),0.0573 (0.0836),0.0025 (0.1195),-0.0175 (0.1014),-0.0401 (0.0458),-0.0145 (0.0367)
phi13,NA,NA,NA,NA,NA,NA,-0.0114 (0.0319),-0.0085 (0.0317),-0.0102 (0.0318)
phi14,NA,NA,NA,NA,NA,NA,-0.1015 (0.1011),-0.0304 (0.0455),-0.052 (0.0366)
phi15,NA,NA,NA,NA,NA,NA,0.0306 (0.0323),0.0309 (0.0322),0.0294 (0.0322)
phi16,NA,NA,NA,NA,NA,NA,0.0792 (0.1011),0.0232 (0.0458),0.0291 (0.0366)
phi17,NA,NA,NA,NA,NA,NA,0.0927 (0.0323),NA,0.0936 (0.0322)
